# Import

In [64]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import sklearn
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu

In [65]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
import keras_tuner as kt

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
df_a=pd.read_csv("../content/drive/MyDrive/TherapEase/adult21.csv")

# Preprocessing

In [69]:
need_cols = ['AGEP_A', 'SEX_A', 'BMICAT_A', 'PHSTAT_A', 'MARSTAT_A', 'EDUCP_A', 'SCHCURENR_A', 'LSATIS11R_A', 
             'SAD_A', 'NERVOUS_A', 'RESTLESS_A', 'HOPELESS_A', 'EFFORT_A', 'WORTHLESS_A', 'K6SPD_A', 'SOCSCLPAR_A', 'EMPWRKLSW1_A', 'EMPWHYNOT_A', 'EMPWKHRS3_A', 'EMPWRKFT1_A', 
             'SUPPORT_A', 'CMPSUPPORT_A', 'FAMCARE12M_A', 'ANXEV_A', 'DEPEV_A', 'ANXFREQ_A', 'ANXLEVEL_A', 'DEPFREQ_A','DEPLEVEL_A']

In [70]:
sub_df = df_a[need_cols]

In [71]:
sub_df.columns = ['age', 'sex', 'bmi', 'physical', 'marital', 'edu', 'school', 'life_satisfaction', 
                  'sad', 'nervous', 'restless', 'hopeless', 'effort', 'worthless', 'k6', 'social', 'work_lastweek', 'work_whynot', 'work_time', 'work_time35', 
                  'support', 'support12m', 'famcare', 'anx_ever', 'dep_ever', 'anx_freq', 'anx_level', 'dep_freq', 'dep_level']

In [72]:
sub_df.head()

,age,sex,bmi,physical,marital,edu,school,life_satisfaction,sad,nervous,...,work_time35,support,support12m,famcare,anx_ever,dep_ever,anx_freq,anx_level,dep_freq,dep_level
0,50,1,3,2,5,1,2,8,5,5,...,1.0,2,3,2,2,2,5,NaN,5,NaN
1,53,1,3,2,6,7,2,8,5,5,...,1.0,1,3,1,2,2,4,3.0,5,NaN
2,56,1,3,2,5,8,2,9,4,4,...,1.0,2,3,2,1,2,2,3.0,4,1.0
3,57,2,4,4,7,5,2,8,5,3,...,NaN,3,1,1,1,2,2,3.0,4,3.0
4,25,1,4,3,9,4,8,8,8,8,...,NaN,8,8,8,2,2,8,NaN,8,NaN


In [73]:
for col in sub_df.columns:
    print(col, sub_df[col].value_counts(), sep='\n')
    print(sub_df[col].isna().sum())
    print("\n")

age
85    1032
61     588
60     564
59     560
64     553
      ... 
83     222
19     219
84     194
97      76
99      10
Name: age, Length: 70, dtype: int64
0


sex
2    16102
1    13378
9        1
7        1
Name: sex, dtype: int64
0


bmi
3    9917
4    9225
2    9144
9     733
1     463
Name: bmi, dtype: int64
0


physical
2    10105
3     8350
1     6657
4     3353
5     1004
9        7
7        6
Name: physical, dtype: int64
0


marital
1    12675
7     6056
5     3926
4     2820
8     1850
9      991
2      730
6      426
3        8
Name: marital, dtype: int64
0


edu
8     6968
4     6606
5     4453
9     3284
7     2566
1     2069
10    1149
6     1126
3      645
2      464
99     101
97      51
Name: edu, dtype: int64
0


school
2    26786
1     1729
8      953
7       12
9        2
Name: school, dtype: int64
0


life_satisfaction
10    8717
8     7656
9     5227
7     3698
5     1537
6     1279
4      324
98     314
3      229
0      141
99     120
2      108
1       71
9

In [74]:
sub_df.loc[:, 'anx_level'] = sub_df.loc[:, 'anx_level'].apply(lambda x : 0 if math.isnan(x) else x)
sub_df.loc[:, 'dep_level'] = sub_df.loc[:, 'dep_level'].apply(lambda x : 0 if math.isnan(x) else x)
sub_df.loc[:, 'work_whynot'] = sub_df.loc[:, 'work_whynot'].apply(lambda x : 0 if math.isnan(x) else x)
sub_df.loc[:, 'work_time'] = sub_df.loc[:, 'work_time'].apply(lambda x : 0 if math.isnan(x) else x)
sub_df.loc[:, 'work_time35'] = sub_df.loc[:, 'work_time35'].apply(lambda x : 0 if math.isnan(x) else x)

<ipython-input-74-73a1d62ab99a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df.loc[:, 'anx_level'] = sub_df.loc[:, 'anx_level'].apply(lambda x : 0 if math.isnan(x) else x)
<ipython-input-74-73a1d62ab99a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df.loc[:, 'dep_level'] = sub_df.loc[:, 'dep_level'].apply(lambda x : 0 if math.isnan(x) else x)
<ipython-input-74-73a1d62ab99a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [75]:
for col in sub_df.columns:
    print(col, sub_df[col].value_counts(), sep='\n')
    print(sub_df[col].isna().sum())
    print("\n")

age
85    1032
61     588
60     564
59     560
64     553
      ... 
83     222
19     219
84     194
97      76
99      10
Name: age, Length: 70, dtype: int64
0


sex
2    16102
1    13378
9        1
7        1
Name: sex, dtype: int64
0


bmi
3    9917
4    9225
2    9144
9     733
1     463
Name: bmi, dtype: int64
0


physical
2    10105
3     8350
1     6657
4     3353
5     1004
9        7
7        6
Name: physical, dtype: int64
0


marital
1    12675
7     6056
5     3926
4     2820
8     1850
9      991
2      730
6      426
3        8
Name: marital, dtype: int64
0


edu
8     6968
4     6606
5     4453
9     3284
7     2566
1     2069
10    1149
6     1126
3      645
2      464
99     101
97      51
Name: edu, dtype: int64
0


school
2    26786
1     1729
8      953
7       12
9        2
Name: school, dtype: int64
0


life_satisfaction
10    8717
8     7656
9     5227
7     3698
5     1537
6     1279
4      324
98     314
3      229
0      141
99     120
2      108
1       71
9

In [76]:
sub_df = sub_df[(sub_df['age'] < 97) & (sub_df['sex'] < 7) & (sub_df['bmi'] < 7) & (sub_df['physical'] < 7) & (sub_df['marital'] < 9) & (sub_df['edu'] < 97) & (sub_df['school'] < 7) & 
                (sub_df['life_satisfaction'] < 97) & (sub_df['k6'] < 7) & (sub_df['social'] < 7) & (sub_df['work_lastweek'] < 7) & (sub_df['work_whynot'] < 8) & (sub_df['work_time'] < 97) & (sub_df['work_time35'] < 7) & 
                (sub_df['support'] < 7) & (sub_df['support12m'] < 7) & (sub_df['famcare'] < 7) & (sub_df['anx_ever'] < 7) & (sub_df['dep_ever'] < 7) & (sub_df['anx_freq'] < 7) & (sub_df['dep_freq'] < 7) & (sub_df['anx_level'] < 7) & (sub_df['dep_level'] < 7)]

In [77]:
for col in sub_df.columns:
    print(col, sub_df[col].value_counts(), sep='\n')
    print(sub_df[col].isna().sum())
    print("\n")

age
85    891
61    513
62    503
66    497
59    496
     ... 
20    211
83    201
82    200
19    198
84    173
Name: age, Length: 68, dtype: int64
0


sex
2    14199
1    12172
Name: sex, dtype: int64
0


bmi
3    9058
4    8528
2    8374
1     411
Name: bmi, dtype: int64
0


physical
2    9228
3    7409
1    6037
4    2864
5     833
Name: physical, dtype: int64
0


marital
1    11826
7     5538
5     3615
4     2596
8     1744
2      664
6      385
3        3
Name: marital, dtype: int64
0


edu
8     6364
4     5862
5     4017
9     3020
7     2318
1     1761
10    1045
6      996
3      574
2      414
Name: edu, dtype: int64
0


school
2    24739
1     1632
Name: school, dtype: int64
0


life_satisfaction
10    7907
8     7055
9     4841
7     3356
5     1333
6     1148
4      278
3      196
0      110
2       90
1       57
Name: life_satisfaction, dtype: int64
0


sad
5    20853
4     3178
3     1726
2      424
1      190
Name: sad, dtype: int64
0


nervous
5    15241
4     6149


In [78]:
def map_age(x):
  if x < 30 :
    return 1
  elif x < 45 :
    return 2
  elif x < 60 :
    return 3
  else :
    return 4
def map_marital(x):
  if x in {1,8} :
    return 0
  else :
    return 1
def map_edu(x):
  if x in {0, 1, 2, 3, 4, 5} :
    return 1
  elif x in {6, 7, 8} :
    return 2
  else :
    return 3
def map_work_whynot(x):
  if x in {1, 4, 5, 7} :
    return 1
  elif x in {2, 3, 6} :
    return 2
  else:
    return x
def map_1(x):
  if x == 3:
    return 4
  else:
    return x
def map_2(x):
  if x == 2:
    return 3
  else:
    return x
def map_3(x):
  if x == 4:
    return 2
  else:
    return x
def map_freq(x):
  return (-1*x)+5

In [79]:
sub_df['age'] = sub_df['age'].apply(map_age)
sub_df['marital'] = sub_df['marital'].apply(map_marital)
sub_df['edu'] = sub_df['edu'].apply(map_edu)
sub_df['work_whynot'] = sub_df['work_whynot'].apply(map_work_whynot)
sub_df['support12m'] = sub_df['support12m'].apply(map_1)
sub_df['support12m'] = sub_df['support12m'].apply(map_2)
sub_df['support12m'] = sub_df['support12m'].apply(map_3)
sub_df['dep_level'] = sub_df['dep_level'].apply(map_1)
sub_df['dep_level'] = sub_df['dep_level'].apply(map_2)
sub_df['dep_level'] = sub_df['dep_level'].apply(map_3)
sub_df['anx_level'] = sub_df['anx_level'].apply(map_1)
sub_df['anx_level'] = sub_df['anx_level'].apply(map_2)
sub_df['anx_level'] = sub_df['anx_level'].apply(map_3)
sub_df['dep_freq'] = sub_df['dep_freq'].apply(map_freq)
sub_df['anx_freq'] = sub_df['anx_freq'].apply(map_freq)
sub_df['sex'] = sub_df['sex'].apply(lambda x : 0 if (x == 2) else x)
sub_df['school'] = sub_df['school'].apply(lambda x : 0 if (x == 2) else x)
sub_df['k6'] = sub_df['k6'].apply(lambda x : 0 if (x == 2) else x)
sub_df['work_lastweek'] = sub_df['work_lastweek'].apply(lambda x : 0 if (x == 2) else x)
sub_df['famcare'] = sub_df['famcare'].apply(lambda x : 0 if (x == 2) else x)
sub_df['anx_ever'] = sub_df['anx_ever'].apply(lambda x : 0 if (x == 2) else x)
sub_df['dep_ever'] = sub_df['dep_ever'].apply(lambda x : 0 if (x == 2) else x)

In [80]:
for col in sub_df.columns:
    print(col, sub_df[col].value_counts(), sep='\n')
    print(sub_df[col].isna().sum())
    print("\n")

age
4    10450
2     6378
3     6051
1     3492
Name: age, dtype: int64
0


sex
0    14199
1    12172
Name: sex, dtype: int64
0


bmi
3    9058
4    8528
2    8374
1     411
Name: bmi, dtype: int64
0


physical
2    9228
3    7409
1    6037
4    2864
5     833
Name: physical, dtype: int64
0


marital
0    13570
1    12801
Name: marital, dtype: int64
0


edu
1    12628
2     9678
3     4065
Name: edu, dtype: int64
0


school
0    24739
1     1632
Name: school, dtype: int64
0


life_satisfaction
10    7907
8     7055
9     4841
7     3356
5     1333
6     1148
4      278
3      196
0      110
2       90
1       57
Name: life_satisfaction, dtype: int64
0


sad
5    20853
4     3178
3     1726
2      424
1      190
Name: sad, dtype: int64
0


nervous
5    15241
4     6149
3     3739
2      776
1      466
Name: nervous, dtype: int64
0


restless
5    16872
4     4288
3     3426
2      931
1      854
Name: restless, dtype: int64
0


hopeless
5    23017
4     1696
3     1181
2      277
1     

In [81]:
def anx_score(row):
  return row['anx_level'] * row['anx_freq']
def dep_score(row):
  return row['dep_level'] * row['dep_freq']

In [82]:
sub_df['anx_score'] = sub_df.apply(anx_score, axis=1)
sub_df['dep_score'] = sub_df.apply(dep_score, axis=1)

In [83]:
print('anx_score', sub_df['anx_score'].value_counts(), sep='\n')
print(sub_df['anx_score'].isna().sum())
print('\n')
print('dep_score', sub_df['dep_score'].value_counts(), sep='\n')
print(sub_df['dep_score'].isna().sum())

anx_score
0.0     7665
1.0     5562
2.0     4089
4.0     2157
6.0     2143
3.0     1917
8.0     1502
12.0     892
9.0      444
Name: anx_score, dtype: int64
0


dep_score
0.0     13900
1.0      4852
2.0      3288
4.0      1102
6.0      1053
3.0       967
12.0      438
8.0       435
9.0       336
Name: dep_score, dtype: int64
0


In [84]:
df = sub_df.copy()

In [85]:
df = df.reset_index()
del df['index']
display(df)

,age,sex,bmi,physical,marital,edu,school,life_satisfaction,sad,nervous,...,support12m,famcare,anx_ever,dep_ever,anx_freq,anx_level,dep_freq,dep_level,anx_score,dep_score
0,3,1,3,2,1,1,0,8,5,5,...,2,0,0,0,0,0.0,0,0.0,0.0,0.0
1,3,1,3,2,1,2,0,8,5,5,...,2,1,0,0,1,2.0,0,0.0,2.0,0.0
2,3,1,3,2,1,2,0,9,4,4,...,2,0,1,0,3,2.0,1,1.0,6.0,1.0
3,3,0,4,4,1,1,0,8,5,3,...,1,1,1,0,3,2.0,1,2.0,6.0,2.0
4,2,1,3,1,0,1,0,10,5,5,...,2,0,0,0,0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26366,4,0,3,2,0,3,0,7,3,4,...,2,0,1,0,3,1.0,1,3.0,3.0,3.0
26367,2,0,4,4,1,2,0,8,5,3,...,2,0,1,1,3,2.0,2,2.0,6.0,4.0
26368,4,0,3,2,0,1,0,6,4,3,...,2,0,0,1,4,3.0,2,1.0,12.0,2.0
26369,3,1,2,3,0,1,0,5,4,4,...,2,0,0,0,4,1.0,3,1.0,4.0,3.0


In [86]:
age_ohe = pd.DataFrame(tf.one_hot(df['age'], depth=len(df['age'].unique())+1)[:, 1:])
age_ohe = age_ohe.add_prefix('age_')
bmi_ohe = pd.DataFrame(tf.one_hot(df['bmi'], depth=len(df['bmi'].unique())+1)[:, 1:])
bmi_ohe = bmi_ohe.add_prefix('bmi_')
edu_ohe = pd.DataFrame(tf.one_hot(df['edu'], depth=len(df['edu'].unique())+1)[:, 1:])
edu_ohe = edu_ohe.add_prefix('edu_')
work_whynot_ohe = pd.DataFrame(tf.one_hot(df['work_whynot'], depth=len(df['work_whynot'].unique())+1)[:, 1:-1])
work_whynot_ohe = work_whynot_ohe.add_prefix('work_whynot_')
work_time35_ohe = pd.DataFrame(tf.one_hot(df['work_time35'], depth=len(df['work_time35'].unique())+1)[:, 1:-1])
work_time35_ohe = work_time35_ohe.add_prefix('work_time35_')
df = df.join(age_ohe)
df = df.join(bmi_ohe)
df = df.join(edu_ohe)
df = df.join(work_whynot_ohe)
df = df.join(work_time35_ohe)
display(df)

,age,sex,bmi,physical,marital,edu,school,life_satisfaction,sad,nervous,...,bmi_1,bmi_2,bmi_3,edu_0,edu_1,edu_2,work_whynot_0,work_whynot_1,work_time35_0,work_time35_1
0,3,1,3,2,1,1,0,8,5,5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3,1,3,2,1,2,0,8,5,5,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,3,2,1,2,0,9,4,4,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,3,0,4,4,1,1,0,8,5,3,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2,1,3,1,0,1,0,10,5,5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26366,4,0,3,2,0,3,0,7,3,4,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
26367,2,0,4,4,1,2,0,8,5,3,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
26368,4,0,3,2,0,1,0,6,4,3,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
26369,3,1,2,3,0,1,0,5,4,4,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [87]:
scl = MinMaxScaler()
df['physical'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['physical'])),columns=['physical'])
df['life_satisfaction'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['life_satisfaction'])),columns=['life_satisfaction'])
df['sad'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['sad'])),columns=['sad'])
df['nervous'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['nervous'])),columns=['nervous'])
df['restless'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['restless'])),columns=['restless'])
df['hopeless'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['hopeless'])),columns=['hopeless'])
df['effort'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['effort'])),columns=['effort'])
df['worthless'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['worthless'])),columns=['worthless'])
df['social'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['social'])),columns=['social'])
df['work_time'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['work_time'])),columns=['work_time'])
df['support'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['support'])),columns=['support'])
df['support12m'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['support12m'])),columns=['support12m'])
df['dep_score'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['dep_score'])),columns=['dep_score'])
df['anx_score'] = pd.DataFrame(scl.fit_transform(pd.DataFrame(df['anx_score'])),columns=['anx_score'])
display(df)

,age,sex,bmi,physical,marital,edu,school,life_satisfaction,sad,nervous,...,bmi_1,bmi_2,bmi_3,edu_0,edu_1,edu_2,work_whynot_0,work_whynot_1,work_time35_0,work_time35_1
0,3,1,3,0.25,1,1,0,0.8,1.00,1.00,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3,1,3,0.25,1,2,0,0.8,1.00,1.00,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,3,0.25,1,2,0,0.9,0.75,0.75,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,3,0,4,0.75,1,1,0,0.8,1.00,0.50,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2,1,3,0.00,0,1,0,1.0,1.00,1.00,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26366,4,0,3,0.25,0,3,0,0.7,0.50,0.75,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
26367,2,0,4,0.75,1,2,0,0.8,1.00,0.50,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
26368,4,0,3,0.25,0,1,0,0.6,0.75,0.50,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
26369,3,1,2,0.50,0,1,0,0.5,0.75,0.75,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [88]:
print(df.columns)

Index(['age', 'sex', 'bmi', 'physical', 'marital', 'edu', 'school',
       'life_satisfaction', 'sad', 'nervous', 'restless', 'hopeless', 'effort',
       'worthless', 'k6', 'social', 'work_lastweek', 'work_whynot',
       'work_time', 'work_time35', 'support', 'support12m', 'famcare',
       'anx_ever', 'dep_ever', 'anx_freq', 'anx_level', 'dep_freq',
       'dep_level', 'anx_score', 'dep_score', 'age_0', 'age_1', 'age_2',
       'age_3', 'bmi_0', 'bmi_1', 'bmi_2', 'bmi_3', 'edu_0', 'edu_1', 'edu_2',
       'work_whynot_0', 'work_whynot_1', 'work_time35_0', 'work_time35_1'],
      dtype='object')


In [89]:
new_cols = ['sex', 'physical', 'marital', 'school',
       'life_satisfaction', 'sad', 'nervous', 'restless', 'hopeless', 'effort',
       'worthless', 'k6', 'social',
       'work_time', 'support', 'support12m', 'famcare',
       'anx_ever', 'dep_ever', 'anx_score', 'dep_score', 'age_0', 'age_1', 'age_2',
       'age_3', 'bmi_0', 'bmi_1', 'bmi_2', 'bmi_3', 'edu_0', 'edu_1', 'edu_2',
       'work_whynot_0', 'work_whynot_1', 'work_time35_0', 'work_time35_1']

In [90]:
dff = df[new_cols]
dfa = dff.drop('dep_score',axis=1)
dfd = dff.drop('anx_score',axis=1)
dfd.columns

Index(['sex', 'physical', 'marital', 'school', 'life_satisfaction', 'sad',
       'nervous', 'restless', 'hopeless', 'effort', 'worthless', 'k6',
       'social', 'work_time', 'support', 'support12m', 'famcare', 'anx_ever',
       'dep_ever', 'dep_score', 'age_0', 'age_1', 'age_2', 'age_3', 'bmi_0',
       'bmi_1', 'bmi_2', 'bmi_3', 'edu_0', 'edu_1', 'edu_2', 'work_whynot_0',
       'work_whynot_1', 'work_time35_0', 'work_time35_1'],
      dtype='object')

In [91]:
Xa, ya, Xd, yd = dfa.drop('anx_score',axis=1), dfa['anx_score'], dfd.drop('dep_score',axis=1), dfd['dep_score']

In [92]:
Xa_train, Xa_rem, ya_train, ya_rem = train_test_split(Xa,ya, train_size=0.7)
Xd_train, Xd_rem, yd_train, yd_rem = train_test_split(Xd,yd, train_size=0.7)
Xa_valid, Xa_test, ya_valid, ya_test = train_test_split(Xa_rem, ya_rem, test_size=0.5)
Xd_valid, Xd_test, yd_valid, yd_test = train_test_split(Xd_rem, yd_rem, test_size=0.5)

In [93]:
print(Xa_train.shape), print(ya_train.shape)
print(Xa_valid.shape), print(ya_valid.shape)
print(Xa_test.shape), print(ya_test.shape)
print(Xd_train.shape), print(yd_train.shape)
print(Xd_valid.shape), print(yd_valid.shape)
print(Xd_test.shape), print(yd_test.shape)

(18459, 34)
(18459,)
(3956, 34)
(3956,)
(3956, 34)
(3956,)
(18459, 34)
(18459,)
(3956, 34)
(3956,)
(3956, 34)
(3956,)


(None, None)

# Model construction

In [94]:
def my_model(hp):
    model = Sequential()
    model.add(tf.keras.Input(shape=(34,)))
    layers = hp.Choice('layer', [1,2])
    neurons = hp.Int('neuron', min_value=13, max_value=27, step=2)

    for i in range(layers):
        model.add(Dense(neurons, activation="relu"))
    model.add(Dense(1, activation="linear"))

    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=7000,
        decay_rate=0.9)

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['mse']
    )
    return model

In [95]:
tuner_anx = kt.GridSearch(
    my_model,
    objective='val_mse',
    max_trials = 16,
    overwrite=True,
    directory="test_search_anx",
    project_name="test_project_anx")

tuner_anx.search(
    Xa_train, ya_train, epochs=70,
    validation_data=(Xa_valid, ya_valid),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Trial 16 Complete [00h 00m 21s]
val_mse: 0.026485707610845566

Best val_mse So Far: 0.026365991681814194
Total elapsed time: 00h 06m 25s


In [96]:
best_hps_anx = tuner_anx.get_best_hyperparameters()[0]

print('best number of layers:',best_hps_anx.get('layer'))
print('best number of neurons:',best_hps_anx.get('neuron'))

best number of layers: 1
best number of neurons: 17


In [97]:
tuner_dep = kt.GridSearch(
    my_model,
    objective='val_mse',
    max_trials = 16,
    overwrite=True,
    directory="test_search_dep",
    project_name="test_project_dep")

tuner_dep.search(
    Xd_train, yd_train, epochs=70,
    validation_data=(Xd_valid, yd_valid),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Trial 16 Complete [00h 00m 15s]
val_mse: 0.01619083620607853

Best val_mse So Far: 0.015834655612707138
Total elapsed time: 00h 06m 29s


In [98]:
best_hps_dep = tuner_dep.get_best_hyperparameters()[0]

print('best number of layers:',best_hps_dep.get('layer'))
print('best number of neurons:',best_hps_dep.get('neuron'))

best number of layers: 2
best number of neurons: 23


In [99]:
anx_model = tuner_anx.hypermodel.build(best_hps_anx)
dep_model = tuner_dep.hypermodel.build(best_hps_dep)

In [100]:
anx_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 17)                595       
                                                                 
 dense_4 (Dense)             (None, 1)                 18        
                                                                 
Total params: 613
Trainable params: 613
Non-trainable params: 0
_________________________________________________________________


In [101]:
dep_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 23)                805       
                                                                 
 dense_6 (Dense)             (None, 23)                552       
                                                                 
 dense_7 (Dense)             (None, 1)                 24        
                                                                 
Total params: 1,381
Trainable params: 1,381
Non-trainable params: 0
_________________________________________________________________


In [102]:
anx_model.fit(Xa_train, ya_train, epochs=100, validation_data=(Xa_valid, ya_valid), callbacks=[tf.keras.callbacks.EarlyStopping(patience=20)])

Epoch 1/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0351 - mse: 0.0351 - val_loss: 0.0308 - val_mse: 0.0308
Epoch 2/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0301 - mse: 0.0301 - val_loss: 0.0275 - val_mse: 0.0275
Epoch 3/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0297 - mse: 0.0297 - val_loss: 0.0293 - val_mse: 0.0293
Epoch 4/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0294 - mse: 0.0294 - val_loss: 0.0265 - val_mse: 0.0265
Epoch 5/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0293 - mse: 0.0293 - val_loss: 0.0268 - val_mse: 0.0268
Epoch 6/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0293 - mse: 0.0293 - val_loss: 0.0273 - val_mse: 0.0273
Epoch 7/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0289 - mse: 0.0289 - val_loss: 0.0276 - val_mse: 0.0276
Epoch 8/100
577/577 [==============================] - 1s 1ms/step - 

In [112]:
ya_pred = anx_model.predict(Xa_test)
mse_anx = sklearn.metrics.mean_squared_error(ya_test, ya_pred)
print('mse anx: ',mse_anx)

124/124 [==============================] - 0s 807us/step
mse anx:  0.029686545545672686


In [104]:
dep_model.fit(Xd_train, yd_train, epochs=100, validation_data=(Xd_valid, yd_valid), callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.0170 - val_mse: 0.0170
Epoch 2/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0162 - mse: 0.0162 - val_loss: 0.0166 - val_mse: 0.0166
Epoch 3/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0161 - mse: 0.0161 - val_loss: 0.0169 - val_mse: 0.0169
Epoch 4/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0159 - mse: 0.0159 - val_loss: 0.0170 - val_mse: 0.0170
Epoch 5/100
577/577 [==============================] - 1s 1ms/step - loss: 0.0157 - mse: 0.0157 - val_loss: 0.0169 - val_mse: 0.0169
Epoch 6/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0157 - mse: 0.0157 - val_loss: 0.0174 - val_mse: 0.0174
Epoch 7/100
577/577 [==============================] - 1s 2ms/step - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0164 - val_mse: 0.0164
Epoch 8/100
577/577 [==============================] - 1s 2ms/step - 

In [111]:
yd_pred = dep_model.predict(Xd_test)
mse_dep = sklearn.metrics.mean_squared_error(yd_test, yd_pred)
print('mse dep: ',mse_dep)

124/124 [==============================] - 0s 997us/step
mse dep:  0.016093539160175214


# Export Model

In [115]:
anx_model.save('/content/anx_model.h5')
dep_model.save('/content/dep_model.h5')

In [116]:
from google.colab import files
files.download('/content/anx_model.h5')
files.download('/content/dep_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>